### Step 1: Combine Flow Cell Data

Concatenate FASTQ files from both flow cells into a single file. Check basic statistics like total read count
and file size to confirm successful combination.

In [1]:
!cd reads && sh sample_reads.sh

Starting subsampling at Fri Oct 17 13:15:47 CDT 2025
pigz: pigz: abort: missing parameter after -pabort: missing parameter after -p

cat: write error: Broken pipe
Finished subsampling at Fri Oct 17 13:15:47 CDT 2025
Output file size:
-rw-r--r-- 1 willetse0745 SFU_Users 0 Oct  9 17:24 planaria_test_subset.fastq.gz
Number of reads in output:

gzip: planaria_test_subset.fastq.gz: unexpected end of file
0


Writes 1 million subsampled reads to `/data/groups/wheelenj/sequencing/20250916_M009242/planaria_test_subset.fastq.gz`

### Step 2: Initial Quality Assessment

Run NanoPlot on the combined dataset to assess read length distribution, quality scores, and overall data
characteristics. Optionally check for adapter contamination using Porechop.

In [2]:
!mkdir fastq_planaria
!mkdir fastq_planaria/qc
!NanoPlot --fastq /data/groups/wheelenj/sequencing/20250916_M009242/planaria_test_subset.fastq.gz -o fastq_planaria/qc

mkdir: cannot create directory ‘fastq_planaria’: File exists
mkdir: cannot create directory ‘fastq_planaria/qc’: File exists
This command requires Kaleido v1.0.0 or greater.
Install it using `pip install 'kaleido>=1.0.0'` or `pip install 'plotly[kaleido]'`."



In [4]:
!multiqc fastq_planaria/.

^C


### Step 3: Read Filtering and Subsampling

Use Filtlong to filter reads based on quality and length, targeting 100-150 GB of high-quality data
(approximately 50-100x coverage). Filter parameters: minimum length 1000 bp, minimum mean quality 8,
keep best 20-25% of reads.

In [5]:
!mkdir trimmed_fastq
!filtlong --min_length 1000 --keep_percent 80 --min_mean_q 8 /data/groups/wheelenj/sequencing/20250916_M009242/planaria_test_subset.fastq.gz | gzip > trimmed_fastq/trimmed_reads.fastq.gz


Scoring long reads
  1,000,000 reads (6,122,479,215 bp)eads (14,224,098 bp)65,036,464 bp)24,402 reads (147,616,755 bp)31,164 reads (187,975,464 bp)53,288 reads (321,771,805 bp)54,443 reads (328,636,749 bp)338,929,413 bp)81,467 reads (493,103,111 bp)88,340 reads (534,047,231 bp)99,333 reads (602,589,618 bp)108,161 reads (656,546,298 bp)705,372,302 bp)807,207,393 bp)133,656 reads (814,066,666 bp)141,893 reads (866,303,240 bp)884,312,746 bp)145,253 reads (887,745,244 bp)961,383,924 bp)1,033,298,822 bp)1,252,943,336 bp)1,300,887,714 bp)1,326,544,524 bp)1,349,658,559 bp)233,334 reads (1,430,139,807 bp)1,513,186,348 bp)250,301 reads (1,535,429,402 bp)255,306 reads (1,566,245,137 bp)288,076 reads (1,766,496,259 bp)307,057 reads (1,882,907,126 bp)1,893,176,769 bp)1,916,298,043 bp)2,157,846,546 bp)359,108 reads (2,202,381,255 bp)2,331,694,961 bp)2,335,120,644 bp)2,360,814,008 bp)387,916 reads (2,377,076,835 bp)389,455 reads (2,386,497,615 bp)399,748 reads (2,448,998,695 bp)2,587,673,997 bp)431

### Step 4: Post-Filtering Quality Control

Run NanoPlot on the combined dataset to assess read length distribution, quality scores, and overall data
characteristics. Optionally check for adapter contamination using Porechop.

In [ ]:
!NanoPlot --fastq trimmed_fastq/trimmed_reads.fastq.gz -o trimmed_fastq/qc --plots kde hex